In [1]:
pip install konlpy

Defaulting to user installation because normal site-packages is not writeable
   ---------------------------------------- 0.0/19.4 MB ? eta -:--:--
   ----------------------------- ---------- 14.2/19.4 MB 79.8 MB/s eta 0:00:01
   ---------------------------------------- 19.4/19.4 MB 67.4 MB/s eta 0:00:00

   ---------------------------------------- 0/2 [JPype1]
   -------------------- ------------------- 1/2 [konlpy]
   -------------------- ------------------- 1/2 [konlpy]
   ---------------------------------------- 2/2 [konlpy]

Note: you may need to restart the kernel to use updated packages.


In [5]:
from selenium import webdriver    
from selenium.webdriver.chrome.options import Options
from bs4 import BeautifulSoup
from openpyxl import Workbook
from konlpy.tag import Okt 
from collections import Counter
import time 

# 크롬 드라이버 옵션 설정
options = Options()
options.add_argument("--no-sandbox")    # 샌드박스 모드 비활성화
options.add_argument("--disable-dev-shm-usage")    # 공유 메모리 사용 비활성화
options.add_argument("--headless")    # 브라우저 창을 띄우지 않게

# 크롬 드라이버 실행 (옵션 적용)
driver = webdriver.Chrome(options=options)

url = "https://www.kurly.com/goods/1000078028"
test = driver.get(url)
time.sleep(5)

soup = BeautifulSoup(driver.page_source, "html.parser")
# print(soup)
reviews = soup.select("p.css-y49dcn.e36z05c13")

#리스트 컴프리헨션으로 리뷰 태그에서 텍스트만 추출하고 공백 제거
# review_lists는 리뷰 텍스트들의 리스트가 됨
review_lists = [review.get_text().strip() for review in reviews]    #

driver.quit()

# 엑셀 워크북 생성
wb = Workbook()
ws = wb.active
ws.title = "Kurly Reviews"
ws.append(["리뷰내용"])

# 리뷰 텍스트 하나씩 엑셀에 행으로 추가
for item in review_lists :
    ws.append([item])   # 한 행에 리스트 형태로 넣음, 대괄호는 엑셀에 행 단위 데이터 입력용

# 형태소 분석기 Okt객체 생성 (한국어 명사 추출용)
okt = Okt()
all_text = " ".join(review_lists)   # 리스트의 리뷰들을 한 문장으로 합침 (공백기준)
nouns = okt.nouns(all_text)   # 명사만 추출해서 리스트로 받음
fiiltered_nouns = [n for n in nouns if len(nouns) > 1]   # 길이가 1보다 큰 명사만 필터링
count = Counter(fiiltered_nouns)   # 단어별 등장 빈도 계산 

ws_keywords = wb.create_sheet("Keyword Frequency")   # 새로운 시트에 키워드 빈도수 저장
ws_keywords.append(["키워드","빈도수"])   # 제목 행추가
for word, freq in count.most_common(30) :    # 빈도수 높은 순서대로 30개 단어와 빈도수 엑셀에 추가
    ws_keywords.append([word, freq])

wb.save("Kurly Reviews.xlsx")
print("엑셀 저장 완료 : Kurly Reviews.xlsx") 

엑셀 저장 완료 : Kurly Reviews.xlsx


In [16]:
from openpyxl import load_workbook   # 기존 엑셀파일 열기 위한 함수
from konlpy.tag import Okt   # 한국어 자연어 처리 라이브러리, 형태소 분석 (키워드 분석 활용)
from collections import Counter
  
wb = load_workbook("연관키워드 20250722 1448.xlsx")   # 기존 저장된 엑셀 파일 불러옴
ws = wb.active   # 현재 활성화된 첫번쨰 시트 사용

# 각 행(row)에서 첫번째 셀 (row[0])만 추출
# 아래 조건에 해당하는 모든 행들에 대해 반복하면서 엑셀 시트의 여러 행을 반복처리
# 2행부터 시작 1열(A열)까지만 읽기 + 셀 값만 가져옴 if 첫 번째 셀 값이 비어 있지 않은 경우에
texts = [row[0] for row in ws.iter_rows(min_row=2, max_col=1, values_only=True) if row[0]]

okt = Okt()   # Okt 형태소 분석기 객체 생성
all_text = " ".join(texts)   # texts 리스트에 들어있는 문자열 공백 기준으로 하나의 문자열로 만듦
nouns = okt.nouns(all_text)   # all_text에서 명사들만 추출하여 리스트로 반환
filtered_nouns = [n for n in nouns if len(n) > 1]   # 
count = Counter(filtered_nouns)

ws_keywords = wb.create_sheet("키워드 전체분석")
ws_keywords.append(["명사", "빈도수"])
for word, freq in sorted(count.items(), key=lambda x:x[1], reverse=True) :
    ws_keywords.append([word, freq])

wb.save("연관키워드_분석결과.xlsx")
print("완료 : 전체 키워드 분석 결과 저장됨")

완료 : 전체 키워드 분석 결과 저장됨


In [25]:
# 실습
from openpyxl import load_workbook
from konlpy.tag import Okt
from collections import Counter

wb = load_workbook("연관키워드 20250722 1550.xlsx")
ws = wb.active

texts = [row[0] for row in ws.iter_rows(min_row=2, max_col=1, values_only=True) if row[0]]

okt=Okt()
all_text = " ".join(texts)
nouns = okt.nouns(all_text)
filtered_nouns = [n for n in nouns if len(n) > 1]
count = Counter(filtered_nouns)

ws_keywords = wb.create_sheet("키워드 전체분석")
ws_keywords.append(["명사", "빈도수"])
for word, freq in sorted(count.items(), key=lambda x:x[1], reverse = True) :
    ws_keywords.append([word, freq])

wb.save("연관키워드_분석결과 수빈.xlsx")
print("완료 : 전체 키워드 분석 결과 저장됨")

완료 : 전체 키워드 분석 결과 저장됨


In [30]:
from openpyxl import load_workbook   # 기존 엑셀파일 열기 위한 함수
from konlpy.tag import Okt   # 한국어 자연어 처리 라이브러리, 형태소 분석 (키워드 분석 활용)
from collections import Counter
import re
  
wb = load_workbook("연관키워드 20250722 1448.xlsx")   # 기존 저장된 엑셀 파일 불러옴
ws = wb.active   # 현재 활성화된 첫번쨰 시트 사용

# 각 행(row)에서 첫번째 셀 (row[0])만 추출
# 아래 조건에 해당하는 모든 행들에 대해 반복하면서 엑셀 시트의 여러 행을 반복처리
# 2행부터 시작 1열(A열)까지만 읽기 + 셀 값만 가져옴 if 첫 번째 셀 값이 비어 있지 않은 경우에
texts = [row[0] for row in ws.iter_rows(min_row=2, max_col=1, values_only=True) if row[0]]

okt = Okt()   # Okt 형태소 분석기 객체 생성
all_text = " ".join(texts)   # texts 리스트에 들어있는 문자열 공백 기준으로 하나의 문자열로 만듦

nouns_tokens = okt.nouns(all_text)    # 전체 텍스트에 명사만 추출

regex_tokens = re.findall(r'[A-Za-z]*\d+[A-Za-z]+|\d+[A-Za-z]*',all_text)   # 정규 표현식 숫자+영문 조합 추출

valid_one_letter = {"금", "은", "동"}   # 의미있는 한글자 단어들 허용
stopwords = {"사용", "정도", "조금", "제품", "때문", "생각", "만족", "구매", "다음"}   # 분석에서 제외할 용어 설정

combined_tokens = nouns_tokens + regex_tokens
filtered_tokens = [t for t in combined_tokens if (len(t) > 1 or t in valid_one_letter) and t not in stopwords]   #

count = Counter(filtered_tokens)

ws_keywords = wb.create_sheet("키워드 전체분석")
ws_keywords.append(["명사", "빈도수"])
for word, freq in sorted(count.items(), key=lambda x:x[1], reverse=True) :
    ws_keywords.append([word, freq])

wb.save("연관키워드_분석결과.xlsx")
print("완료 : 전체 키워드 분석 결과 저장됨")

완료 : 전체 키워드 분석 결과 저장됨
